## Goals:

- load the G from layers for miniaste
- create the G_advh from offline monthly avg.
- load the gateways (make sure indices)
- verify this way
- now verify the M term

## setup

In [1]:
# auto-reload edited modules
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, "/home/mmurakami/crios_backups/an_helper_functions")

# run the script into the curre nt kernel (vars/functions become available)
%run -i "/home/mmurakami/crios_backups/an_helper_functions/prep_grid_aste_90.py"

(450, 90)
RAC2d (40500,)
(100,)


In [2]:
dirroot = "/scratch3/atnguyen/aste_90x150x60/"
dirgrid = dirroot + "GRID_real8/"
dirgridnb = dirroot + "GRID_noblank/"
runstr= "run_c68v_heffmosm3x_layers_lessmem1_viscAHp5em2_it0000_pk0000000001/"
layers_path = dirroot + runstr
extL = "LAYERS"
dirmask = dirroot + "run_template/input_maskTransport/"
dirbudg = layers_path + "diags/BUDG/"
dirdiags = dirbudg
dirstate = layers_path + "diags/STATE/"
dirlayers = layers_path + "diags/LAYERS/"
dirtrsp = layers_path + "diags/TRSP/"

In [3]:
# maybe we can dump a file to load the aste90 grid so I can have all the values

## Make sure we have the bins

In [4]:
filename = "layers2TH"
boundsT = rdmds(layers_path + filename)
binsTH_edges = boundsT.reshape(boundsT.shape[0])
binsTH_centers = (binsTH_edges[:-1] + binsTH_edges[1:])/2
nT = binsTH_edges.shape[0]-1

filename = "layers1SLT"
boundsS = rdmds(layers_path + filename)
binsSLT_edges = boundsS.reshape(boundsS.shape[0])
binsSLT_centers = (binsSLT_edges[:-1] + binsSLT_edges[1:])/2
nS = binsSLT_edges.shape[0]-1

binwidthT = binsTH_edges[1:] - binsTH_edges[:-1]
binwidthS = binsSLT_edges[1:] - binsSLT_edges[:-1]

binwidthT1 = (binwidthT[:-1] + binwidthT[1:])/2
binwidthS1 = (binwidthS[:-1] + binwidthS[1:])/2

binmidT = ((boundsT[:-1] + boundsT[1:])/2).reshape(nT)
binmidS = ((boundsS[:-1] + boundsS[1:])/2).reshape(nT)

## Load the G_T and G_S terms

In [5]:
# we need to also create the theta and salt bins for this area
tsstr = np.array(["0000000002","0000000003"])
tsstr = np.array(["0000000003","0000000004"])

t2 = int(tsstr[1]) # for the offline version

In [6]:
ffac=1e-6

## Create the ADVh terms from the salt and temp

In [9]:
from create_aste90_layers import create_layers_totalTHETA,create_layers_totalSALT

ImportError: cannot import name 'create_layers_totalSALT' from 'create_aste90_layers' (/home/mmurakami/crios_backups/ASTE_270/Pemberton_BarentsSpaper/2026_BarentsS/create_aste90_layers.py)

In [10]:
Msum, dF_Tnew = create_layers_totalTHETA(tsstr,mygrid,myparms,dirdiags,dirstate,layers_path,mymsk,nz,ny,nx,nfx,nfy)

NameError: name 'mymsk' is not defined

## verify these manually

In [112]:
def _bincount_sum_with_nan(idx, vals, nout):
    """
    NaN-aware per-bin sum:
    if *all* entries in a bin are NaN, that bin returns NaN;
    otherwise NaNs are ignored and finite values are summed.
    """
    # track counts of non-nan contributions
    finite = np.isfinite(vals)
    sums   = np.bincount(idx[finite], vals[finite], minlength=nout).astype(float)
    counts = np.bincount(idx[finite], None, minlength=nout).astype(float)
    out = sums
    out[counts == 0] = np.nan
    return out

## Create the M term from the gates

We need to find the previous notebook where I marked the gates. Not sure what I called it.

try_again_BarentsSwhole.ipynb

In [113]:
## I think the best way to do this is with the ADV_TH term

In [114]:
labels = np.array(["ADVh","ADVr","DFh","DFr","surf","kpp","tend"])

s = 0 
print("gates       ",np.nansum(np.nansum(Msum,axis=0))/np.nansum(binwidthT1),"m^3/s")
for i in range(6):
    at = np.nansum(dF_Tnew[i]) / np.nansum(binwidthT1)
    print(labels[i],"       ",at,"m^3/s")
    s += at
print()
print("total volume tend RHS: ",s,"m^3/s")
print("total volume tend LHS: ",np.nansum(dF_Tnew[-1]/1.9131428585e3) / np.nansum(binwidthS1),"m^3/s")

gates        265872.3371250672 m^3/s
ADVh         265872.3371250672 m^3/s
ADVr         -2.2253824948265377e-11 m^3/s
DFh         106.12036325954855 m^3/s
DFr         1.7803059958612301e-10 m^3/s
surf         -646325.4887748167 m^3/s
kpp         5.563456237066344e-12 m^3/s

total volume tend RHS:  -380347.0312864898 m^3/s
total volume tend LHS:  -380347.0312333998 m^3/s


## Load the salt terms and the gates from SLT

In [79]:
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["ADVx_SLT","ADVy_SLT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVx_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVy_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])

# now 3d zfluxes
file_name = "budg3d_zflux_set2"
meta_budg3d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set1["fldList"])
varnames = np.array(["ADVr_SLT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVr_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVr_SLT = ADVr_SLT.reshape(nz,ny,nx)

hf = mygrid['hFacC']

ADV_hconv = calc_UV_conv_mod(nfx, nfy,get_aste_faces(ADVx_SLT.reshape(nz, ny, nx), nfx, nfy),get_aste_faces(ADVy_SLT.reshape(nz, ny, nx), nfx, nfy))
ADV_hconv = ADV_hconv * hf   # degC·m^3/s at cell centers (matches: ff.DFh = ff.DFh .* hf)
ADVhS = ADV_hconv

trWtopADV = -(ADVr_TH)

ADVrS = np.zeros((nz,ny,nx),dtype=float)
ADVrS[:-1,:,:] = (trWtopADV[:-1] - trWtopADV[1:])  # this is not the way we did it in the original code but this is the way An has done it so we try

In [80]:

# --- reshape to 3D ---
ADVx_SLT = ADVx_SLT.reshape((nz, ny, nx))   # advective heat flux on x-faces
ADVy_SLT = ADVy_SLT.reshape((nz, ny, nx))   # advective heat flux on y-faces
SALT   = SALT.reshape((nz, ny, nx))     # cell-centered temperature

eps = 1e-6  # to avoid divide-by-zero in very cold cells

ADVx_vol = np.zeros_like(ADVx_SLT)
mask_x   = np.isfinite(theta_x) & (np.abs(theta_x) > eps)
ADVx_vol[mask_x] = ADVx_SLT[mask_x] #/ theta_x[mask_x]

ADVy_vol = np.zeros_like(ADVy_TH)
mask_y   = np.isfinite(theta_y) & (np.abs(theta_y) > eps)
ADVy_vol[mask_y] = ADVy_SLT[mask_y] #/ theta_y[mask_y]

# bolus
ADVx_vol = np.zeros_like(ADVx_TH)
mask_x   = np.isfinite(theta_x) & (np.abs(theta_x) > eps)
ADVx_vol[mask_x] = (ADVx_SLT[mask_x]) #/ theta_x[mask_x]

ADVy_vol = np.zeros_like(ADVy_TH)
mask_y   = np.isfinite(theta_y) & (np.abs(theta_y) > eps)
ADVy_vol[mask_y] = (ADVy_SLT[mask_y]) #/ theta_y[mask_y]

# ------------------------------------------------------------------
# 3. Build gateway transports using volume fluxes at the faces
#    Sign convention: comments assume "positive into basin"
# ------------------------------------------------------------------

# ADVx_vol = ADVx_TH
# ADVy_vol = ADVy_TH

# ---- BSO ----
ADV_west = np.zeros((nz, ny, nx))
y_bso_all = np.array([], dtype=int)
x_bso_all = np.array([], dtype=int)

# horizontal faces (u-faces)
for j, i in zip(y_bsoh, x_bsoh):
    # flux through x-face at (j,i) mapped into cell (j,i)
    ADV_west[:, j, i] += ADVx_vol[:, j, i]    # + into basin
    y_bso_all = np.append(y_bso_all, j)
    x_bso_all = np.append(x_bso_all, i)

# vertical faces (v-faces)
for j, i in zip(y_bsov, x_bsov):
    # flux through y-face at (j,i) mapped into cell (j-1,i)
    ADV_west[:, j-1, i] -= ADVy_vol[:, j, i]  # sign chosen so + into basin
    y_bso_all = np.append(y_bso_all, j-1)
    x_bso_all = np.append(x_bso_all, i)

# ---- FJNZ ----
ADV_FJNZ = np.zeros((nz, ny, nx))
y_fjnz_all = np.array([], dtype=int)
x_fjnz_all = np.array([], dtype=int)

for j, i in zip(y_fjnzv, x_fjnzv):
    # x-face at (j,i) mapped into (j, i-1), + into basin
    ADV_FJNZ[:, j, i-1] -= ADVx_vol[:, j, i]
    y_fjnz_all = np.append(y_fjnz_all, j)
    x_fjnz_all = np.append(x_fjnz_all, i-1)

# ---- SPFJ (NZ exit) ----
ADV_SPFJ = np.zeros((nz, ny, nx))
y_spfj_all = np.array([], dtype=int)
x_spfj_all = np.array([], dtype=int)


# # CHANGED
for j,i in zip(y_spfjv2,x_spfjv2):
    ADV_SPFJ[:, j-1, i] -= ADVy_vol[:, j, i]
    y_spfj_all = np.append(y_spfj_all, j-1)
    x_spfj_all = np.append(x_spfj_all, i)

for j,i in zip(y_spfjh2,x_spfjh2):
    ADV_SPFJ[:, j, i-1] -= ADVx_vol[:, j, i]
    y_spfj_all = np.append(y_spfj_all, j)
    x_spfj_all = np.append(x_spfj_all, i-1)
    
for j,i in zip(y_spfjb2,x_spfjb2):
    ADV_SPFJ[:, j, i-1] -= ADVx_vol[:, j, i]
    y_spfj_all = np.append(y_spfj_all, j)
    x_spfj_all = np.append(x_spfj_all, i-1)
    ADV_SPFJ[:, j-1, i] -= ADVy_vol[:, j, i]
    y_spfj_all = np.append(y_spfj_all, j-1)
    x_spfj_all = np.append(x_spfj_all, i)

# ---- NZRU (small Russia gate) ----
ADV_NZRU = np.zeros((nz, ny, nx))
y_nzru_all = np.array([], dtype=int)
x_nzru_all = np.array([], dtype=int)

for j, i in zip(y_nzruv, x_nzruv):
    ADV_NZRU[:, j, i-1] -= ADVx_vol[:, j, i]   # + into basin
    y_nzru_all = np.append(y_nzru_all, j)
    x_nzru_all = np.append(x_nzru_all, i-1)


In [81]:
ADV_west_flat   = ADV_west.ravel()
ADV_fjnz_flat   = ADV_FJNZ.ravel()
ADV_spfj_flat   = ADV_SPFJ.ravel()
ADV_nzru_flat   = ADV_NZRU.ravel()


# per-bin sums with NaN-propagation
ADVh_BSO = _bincount_sum_with_nan(idx_midS, ADV_west_flat[valid_midS], nTm1)
ADVh_FJNZ = _bincount_sum_with_nan(idx_midS, ADV_fjnz_flat[valid_midS], nTm1)
ADVh_SPFJ = _bincount_sum_with_nan(idx_midS, ADV_spfj_flat[valid_midS], nTm1)
ADVh_NZRU = _bincount_sum_with_nan(idx_midS, ADV_nzru_flat[valid_midS], nTm1)


# edge-based G (m^3/s): divide by edge binwidths
# this is not correct because we want to divide by Face T
G_BSO = ADVh_BSO / binwidthS1
G_FJNZ = ADVh_FJNZ / binwidthS1
G_SPFJ = ADVh_SPFJ / binwidthS1
G_NZRU = ADVh_NZRU / binwidthS1

In [82]:
# define the other terms (all)

# cool, do the same for diff


## do the advective convergence
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["DFxE_SLT","DFyE_SLT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
DFxE_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
DFyE_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])


# now 3d zfluxes
file_name = "budg3d_zflux_set2"
meta_budg3d_zflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set2["fldList"])
varnames = np.array(["DFrE_SLT","DFrI_SLT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
DFrE_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
DFrI_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
DFrE_SLT = DFrE_SLT.reshape(nz,ny,nx)
DFrI_SLT = DFrI_SLT.reshape(nz,ny,nx)

DF_hconv = calc_UV_conv_mod(nfx, nfy,get_aste_faces(DFxE_SLT.reshape(nz, ny, nx), nfx, nfy),get_aste_faces(DFyE_SLT.reshape(nz, ny, nx), nfx, nfy))
DF_hconv = DF_hconv * hf   # degC·m^3/s at cell centers (matches: ff.DFh = ff.DFh .* hf)
DFhS = DF_hconv

trWtopDF = -(DFrE_SLT+DFrI_SLT)

DFrS = np.zeros((nz,ny,nx),dtype=float)
DFrS[:-1,:,:] = (trWtopDF[:-1] - trWtopDF[1:])

# load the surface terms
# read fluxes
file_name = 'budg2d_zflux_set1'
meta_budg2d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg2d_zflux_set1["fldList"])
varnames = np.array(["oceSPflx","SFLUX"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
oceSPflx,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
SFLUX,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
oceSPflx = oceSPflx.reshape(ny,nx)
SFLUX = SFLUX.reshape(ny,nx)

# read relax and salt mass
file_name = "budg2d_zflux_set2"
meta_budg2d_zflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg2d_zflux_set2["fldList"])
varnames = np.array(["oceSflux","WSLTMASS"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    if len(irec[0]) > 0:
        recs = np.append(recs, irec[0][0])
oceSflux,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
WSLTMASS,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
oceSflux = oceSflux.reshape(ny,nx)
WSLTMASS = WSLTMASS.reshape(ny,nx)

# read kpp tend and from 3d zflux
file_name = "budg3d_kpptend_set1"
meta_budg3d_kpptend_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_kpptend_set1["fldList"])
varnames = np.array(["oceSPtnd","KPPg_SLT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    if len(irec[0]) > 0:
        recs = np.append(recs, irec[0][0])
oceSPtnd,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
KPPg_SLT,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
oceSPtnd = oceSPtnd.reshape(nz,ny,nx)
KPPg_SLT = KPPg_SLT.reshape(nz,ny,nx)

RAC3 = np.tile(RAC[np.newaxis,:,:],(nz,1,1))
sptop = mk3D_mod(oceSPflx,oceSPtnd) - np.cumsum(oceSPtnd, axis=0)        # we include this in our zconv_top term
sptop = sptop * RAC3        # g/s

zconv_top_salt = (SFLUX + oceSPflx) * RAC               # g/s

def surface_contrib_JT(zconv_top_salt, sptop, rho, fill_last=0.0):
    """
    zconv_top_heat: (ny, nx)
    swtop:          (nz, ny, nx)
    rcp:            scalar
    fill_last:      value for bottom slice (k = nz-1), usually 0.0 or np.nan
    returns:
      JsurfT:       (nz, ny, nx)  # Sv / PSU
    """
    nz, ny, nx = sptop.shape

    eS = zconv_top_salt.reshape(1, ny, nx)  # (1,ny,nx) for broadcast

    J = np.empty_like(sptop, dtype=float)

    # k = 0: (eT - fT[1]) / rcp / dT / dS * 1e-6
    J[0] = (eS[0] - sptop[1]) / rho if np.ndim(binwidthS)==0 else \
           (eS[0] - sptop[1]) / rho

    # 1 .. nz-2: -(fT[k+1]-fT[k]) / rcp / dT / dS * 1e-6
    J[1:nz] = -(sptop[1:nz] - sptop[0:nz-1]) / rho

    # bottom slice (k = nz-1): no k+1; choose your boundary convention
    J[-1] = fill_last
    return J

Ft_surftest = surface_contrib_JT(zconv_top_salt,sptop,myparms['rhoconst'])    # this is in PSU.m^3/s

# do the vertical convergence for KPP
trWtopKPP = -(KPPg_SLT)         # PSU.m^3/s

tmpkpp = np.full((nz,ny,nx),np.nan)
tmpkpp[:-1,:,:] = trWtopKPP[:-1] - trWtopKPP[1:]

file_name = 'budg3d_snap_set2'
meta_budg3d_snap_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_snap_set2["fldList"])
varnames = np.array(["SALTDR"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])


SALTDR = np.full((len(tsstr),nz,ny,nx),np.nan)
for i in range(len(tsstr)):
    thisSALTDR,its,meta = rdmds(os.path.join(dirdiags, file_name),int(tsstr[i]),returnmeta=True,rec=recs[0])
    thisSALTDR = thisSALTDR.reshape(nz,ny,nx)
    SALTDR[i] = thisSALTDR

SALTDR =  (SALTDR[1, :, :,:] - SALTDR[0, :,:, :]) / 1    # PSU.m/s
#print(np.nansum(SALTDR),dt)

tmptend = (SALTDR - 0) * mk3D_mod(RAC,SALTDR)    # PSU.m/s * m^2 = PSU.m^3/s

In [83]:
# redefine all the terms as a list from how we did before

# define the ADVh total for this mymsk2
G_S_offline_new = np.zeros((7, nS-1))
dF_Snew = np.zeros((7, nS-1))
Lijnew = np.zeros((7, nS-1), dtype=int)

# also mask these by mymsk3
# flatten the 3D arrays along all dimensions, as MATLAB’s tmp(:) does
S_flat    = np.ravel(SALT*hf* mymsk3d, order='F')
ADVh_flat = np.ravel(ADVhS*hf* mymsk3d,  order='F')
ADVr_flat = np.ravel(ADVrS*hf* mymsk3d,  order='F')
DFh_flat = np.ravel(DFhS*hf* mymsk3d,  order='F')
DFr_flat = np.ravel(DFrS*hf* mymsk3d,  order='F')
surf_flat = np.ravel(Ft_surftest*hf* mymsk3d,  order='F')
kpp_flat = np.ravel(tmpkpp*hf* mymsk3d,  order='F')
tend_flat = np.ravel(tmptend*hf* mymsk3d,  order='F')

for i in range(nT-1):
    # MATLAB: ij = find(tmp(:) >= bbb.binmidT(i) & tmp(:) < bbb.binmidT(i+1))
    ij = np.where((S_flat >= binmidS[i]) & (S_flat < binmidS[i + 1]))[0]
    Lijnew[0, i] = len(ij)

    if len(ij) > 0:
        # MATLAB: dF_Tnew(4,i)=sum(ff.advh(ij)); dF_Tnew(5,i)=sum(ff.advr(ij));
        dF_Snew[0, i] = np.nansum(ADVh_flat[ij])
        dF_Snew[1, i] = np.nansum(ADVr_flat[ij])
        dF_Snew[2, i] = np.nansum(DFh_flat[ij])
        dF_Snew[3, i] = np.nansum(DFr_flat[ij])
        dF_Snew[4, i] = np.nansum(surf_flat[ij])
        dF_Snew[5, i] = np.nansum(kpp_flat[ij])
        dF_Snew[6, i] = np.nansum(tend_flat[ij])

# MATLAB: G_T_offline_new = dF_Tnew ./ repmat(bbb.binwidthT1,[6 1])
G_S_offline_new = dF_Snew / binwidthS1[None, :]

In [84]:
print(np.nansum(ADVhS*mymsk3d))  # but this is in degC.m^3/s
print()
print(np.nansum(dF_Snew[0]) / np.nansum(binwidthT1))
print(np.nansum((ADVh_BSO + ADVh_FJNZ + ADVh_SPFJ + ADVh_NZRU)) / np.nansum(binwidthT1))  # in m^3/s

-4392889.650053502

-104967.49456513517
-104967.49456513507


In [85]:
s = 0 
print("gates       ",np.nansum((ADVh_BSO + ADVh_FJNZ + ADVh_SPFJ + ADVh_NZRU))/np.nansum(binwidthS1),"m^3/s")
for i in range(6):
    at = np.nansum(dF_Snew[i]) / np.nansum(binwidthS1)
    print(labels[i],"       ",at,"m^3/s")
    s += at
print()
print("total volume tend RHS: ",s,"m^3/s")
print("total volume tend LHS: ",np.nansum(dF_Snew[-1]/1.8e3) / np.nansum(binwidthS1),"m^3/s")

gates        -111565.45142993073 m^3/s
ADVh         -111565.45142993085 m^3/s
ADVr         1.47828980097695e-12 m^3/s
DFh         30.725026611431016 m^3/s
DFr         -4.157690065247672e-12 m^3/s
surf         -7434.17458396522 m^3/s
kpp         -1.8478622512211875e-13 m^3/s

total volume tend RHS:  -118968.90098728464 m^3/s
total volume tend LHS:  -118968.90098808928 m^3/s


In [86]:
# great, now we can wrap all of this for tsstr2--5

'/scratch3/atnguyen/aste_90x150x60/run_c68v_heffmosm3x_layers_lessmem1_viscAHp5em2_it0000_pk0000000001/diags/BUDG/'